In [1]:
import sqlite3
from datetime import datetime
from itertools import groupby

In [117]:
# Подключение к базе данных (файл будет создан, если не существует)
conn = sqlite3.connect('db/trading_bot.db')

# Создание курсора
cursor = conn.cursor()

# Создание таблицы сделок
cursor.execute('''
CREATE TABLE IF NOT EXISTS deals (
    id INTEGER PRIMARY KEY,
    algorithm_name TEXT NOT NULL,
    type INTEGER NOT NULL,
    instrument TEXT NOT NULL,
    datetime DATETIME DEFAULT CURRENT_TIMESTAMP,
    price REAL NOT NULL,
    commission REAL NOT NULL,
    total REAL NOT NULL
)
''')

# Создание индексов
cursor.execute('CREATE INDEX IF NOT EXISTS idx_instrument_datetime ON deals (algorithm_name, datetime)')
cursor.execute('CREATE INDEX IF NOT EXISTS idx_datetime ON deals (datetime)')

# Закрытие соединения
conn.close()

In [86]:
def get_daily_totals():
    conn = sqlite3.connect('db/trading_bot.db')
    cursor = conn.cursor()
    
    # Получаем аггрегированные данные по дням и algorithm_name
    cursor.execute('''
    SELECT 
        algorithm_name, 
        strftime('%Y-%m-%d', datetime) AS day, 
        SUM(total) AS daily_total,
        count(*) AS cnt
    FROM deals
    GROUP BY algorithm_name, day
    ORDER BY algorithm_name, day
    ''')
    
    aggregated_results = cursor.fetchall()
    
    # Итерация по аггрегированным результатам и выполнение дополнительного запроса для каждой пары algorithm_name, day
    for algorithm_name, day, daily_total, cnt in aggregated_results:
        cursor.execute('''
        SELECT type, total FROM deals
        WHERE algorithm_name = ? AND date(datetime) = date(?)
        ORDER BY datetime DESC
        LIMIT 1
        ''', (algorithm_name, day))
        
        last_operation_type, last_total = cursor.fetchone()
        
        # Если тип последней операции равен 1, вычитаем её total из итогового результата
        if last_operation_type == 1:           
            adjusted_total = daily_total - last_total
            print(f"День: {day}, {algorithm_name}, "
                  f"Подытог: {round(adjusted_total, 2)}, "
                  f"Записей: {cnt}")
        else:
            print(f"День: {day}, {algorithm_name}, "
                  f"Итог: {round(daily_total, 2)}, "
                  f"Записей: {cnt}")
    
    conn.close()

In [54]:
def add_deal(algorithm_name, type, instrument, price, commission, total):
    conn = sqlite3.connect('db/trading_bot.db')
    cursor = conn.cursor()
    cursor.execute('''
    INSERT INTO deals (algorithm_name, type, instrument, price, commission, total)
    VALUES (?, ?, ?, ?, ?, ?)
    ''', (algorithm_name, type, instrument, price, commission, total))
    conn.commit()
    conn.close()

In [3]:
# добавление записи
# add_deal(f"RNFT_v2_c4_s2.py", 2, 'RNFT', 162.6, 0.9, 162.6-0.9)

In [112]:
def get_hourly_totals(date=None):
    if date is None:
        date = datetime.now().strftime('%Y-%m-%d')  # Текущая дата в формате строки

    conn = sqlite3.connect('db/trading_bot.db')
    cursor = conn.cursor()

    # Получаем данные за указанный день
    cursor.execute('''
    SELECT 
        algorithm_name,
        strftime('%H', datetime) AS hour,
        total,
        type
    FROM deals
    WHERE date(datetime) = ?
    ORDER BY algorithm_name, hour, datetime, id
    ''', (date,))

    results = cursor.fetchall()

    # группировка по имени алгоритма
    for algorithm_name, algorithm_group in groupby(results, key=lambda x: x[0]):
        algorithm_list = list(algorithm_group)
        
        print(f"Algorithm: {algorithm_name}")
        
        sum_price = 0
        prev_show = None
        # группировка часам
        for hour, hour_group in groupby(algorithm_list, key=lambda x: x[1]):
            hour_list = list(hour_group)
            
            hour_sum_price = 0
            operation_type = 0
            operation_total = 0 
            for _, _, operation_total, operation_type in hour_list:
                hour_sum_price += operation_total
            
            diff = hour_sum_price
            sum_price += hour_sum_price
            
            show_sum = sum_price
            # откидываем последнюю операцию, но только для локального вывода
            if operation_type == 1:
                show_sum -= operation_total
                # diff -= operation_total
                
            print(f"  {int(hour) + 3}: {round(show_sum, 2)} "
                  f"({round(show_sum - prev_show, 2) if prev_show is not None else '0'})")

            prev_show = show_sum                
            
    conn.close()

In [115]:
get_daily_totals()

День: 2024-03-13, v2_t10.py, Итог: -4.92, Записей: 58
День: 2024-03-13, v2_t300.py, Итог: -4.1, Записей: 18
День: 2024-03-13, v2_t300_s4.py, Итог: -0.16, Записей: 24
День: 2024-03-13, v2_t60.py, Подытог: -3.98, Записей: 51
День: 2024-03-13, v2_t60_s4.py, Подытог: -2.95, Записей: 35
День: 2024-03-13, v3_t300.py, Итог: -4.05, Записей: 44
День: 2024-03-13, v3_t300_s4.py, Итог: -2.55, Записей: 34
День: 2024-03-13, v3_t600.py, Итог: -3.46, Записей: 34


In [116]:
get_hourly_totals()  # Для текущего дня
# get_hourly_totals('2023-01-01')  # Для конкретной даты

Algorithm: v2_t10.py
  10: -0.43 (0)
  11: -0.89 (-0.46)
  12: -0.87 (0.02)
  13: -1.1 (-0.23)
  14: -1.0 (0.1)
  15: -1.64 (-0.64)
  16: -3.46 (-1.82)
  17: -4.38 (-0.92)
  18: -4.92 (-0.54)
Algorithm: v2_t300.py
  11: 0.0 (0)
  12: -0.68 (-0.68)
  13: -0.6 (0.08)
  14: -0.46 (0.14)
  15: -0.54 (-0.08)
  16: -2.02 (-1.48)
  17: -2.02 (0.0)
  18: -4.1 (-2.08)
Algorithm: v2_t300_s4.py
  11: -0.08 (0)
  12: -0.08 (0.0)
  13: 0.96 (1.04)
  14: 1.5 (0.54)
  15: 0.84 (-0.66)
  16: 0.58 (-0.26)
  17: 0.5 (-0.08)
  18: -0.16 (-0.66)
Algorithm: v2_t60.py
  10: -0.56 (0)
  11: -1.02 (-0.46)
  12: -0.9 (0.12)
  13: 0.02 (0.92)
  14: -0.4 (-0.42)
  15: -1.16 (-0.76)
  16: -2.48 (-1.32)
  17: -3.52 (-1.04)
  18: -3.98 (-0.46)
Algorithm: v2_t60_s4.py
  10: -0.16 (0)
  11: -0.34 (-0.18)
  12: -0.72 (-0.38)
  13: -0.17 (0.55)
  14: -0.71 (-0.54)
  15: -1.29 (-0.58)
  16: -2.71 (-1.42)
  17: -2.69 (0.02)
  18: -2.95 (-0.26)
Algorithm: v3_t300.py
  10: -0.28 (0)
  11: -0.54 (-0.26)
  12: -0.52 (0.02)
 